In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup 
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import json
import random

## This code is for extracting the car links from the website

In [ ]:
# def get_browser():
#     return Browser('chrome')

# browser = get_browser()
# target_city = "delhi"
# output_file = f"car_links_{target_city}.txt"
# url = f"https://www.cardekho.com/used-cars+in+{target_city}"

# try:
#     browser.visit(url)
#     time.sleep(3) # Initial load

#     extracted_links = set()
#     no_change_count = 0

#     while no_change_count < 3:
#         previous_count = len(extracted_links)
        
#         # 1. Faster, snappy scrolling
#         # We use a smaller sleep (0.5 to 0.8) between small scrolls
#         for _ in range(8):
#             browser.execute_script("window.scrollBy(0, 1500);")
#             time.sleep(random.uniform(0.5, 0.8)) 
        
#         # 2. Faster "View More" check
#         # .is_element_present_by_text with a low wait_time is very fast
#         if browser.is_element_present_by_text('View More Cars', wait_time=0.5):
#             try:
#                 browser.find_by_text('View More Cars').click()
#                 # Short wait after click for the DOM to update
#                 time.sleep(1.5) 
#             except:
#                 pass

#         # 3. Quick Extraction
#         page_soup = soup(browser.html, 'html.parser')
#         car_containers = page_soup.find_all('div', class_=['titlebox', 'hover'])
        
#         for container in car_containers:
#             link_tag = container.find('a', href=True)
#             if link_tag:
#                 full_link = f"https://www.cardekho.com{link_tag['href']}".split('?')[0]
#                 if "/used-car-details/" in full_link and full_link not in extracted_links:
#                     extracted_links.add(full_link)
#                     with open(output_file, "a") as f:
#                         f.write(full_link + "\n")

#         current_count = len(extracted_links)
#         print(f"Collected: {current_count}")

#         # 4. Smart Exit
#         if current_count > previous_count:
#             no_change_count = 0
#         else:
#             no_change_count += 1
#             # If no progress, wait a tiny bit longer once to see if it's just lag
#             time.sleep(2)

# except Exception as e:
#     print(f"Error: {e}")
# finally:
#     browser.quit()
#     print(f"Done! Final unique links: {len(extracted_links)}")

Collected: 58
Collected: 92
Collected: 140
Collected: 173
Collected: 225
Collected: 283
Collected: 345
Collected: 398
Collected: 452
Collected: 510
Collected: 552
Collected: 598
Collected: 649
Collected: 703
Collected: 748
Collected: 799
Collected: 829
Collected: 885
Collected: 928
Collected: 976
Collected: 1024
Collected: 1068
Collected: 1101
Collected: 1101
Collected: 1101
Collected: 1101
Done! Final unique links: 1101


---

In [3]:
def get_browser():
    return Browser('chrome')

# 1. Load your links
with open("car_links_delhi.txt", "r") as f:
    all_links = [line.strip() for line in f.readlines()]

browser = get_browser()
output_file = "car_dataset_delhi.json"

for index, link in enumerate(all_links):
    try:
        print(f"Scraping {index+1}/{len(all_links)}: {link}")
        browser.visit(link)
        
        # --- Adjusted Timing ---
        # Increased initial wait to 1.5-2s to ensure the button and price load
        time.sleep(random.uniform(.5, 1))
        browser.execute_script("window.scrollTo(0, 600);")
        time.sleep(0.5)

        # 2. Expansion Logic
        try:
            view_all_spec_btn = browser.find_by_text('View all Specifications')
            if view_all_spec_btn:
                browser.execute_script("arguments[0].click();", view_all_spec_btn.first._element)
                # Reduced wait slightly to 1.5s for speed, usually enough for text
                time.sleep(1) 
        except Exception:
            pass

        # 3. Scrape with BeautifulSoup
        page_soup = soup(browser.html, 'html.parser')
        car_data = {"url": link}
        
        # --- PRICE EXTRACTION ---
        # Targeting the div and span seen in your previous screenshots
        price_div = page_soup.find('div', class_='vehiclePrice')
        if price_div:
            price_span = price_div.find('span')
            if price_span:
                car_data["Price"] = price_span.get_text(strip=True)

        # 4. Extract Labels and Values
        spec_items = page_soup.find_all('li', class_='gsc_col-xs-12')
        for item in spec_items:
            label_tag = item.find('div', class_='label')
            value_tag = item.find('span', class_='value-text')
            if label_tag and value_tag:
                car_data[label_tag.get_text(strip=True)] = value_tag.get_text(strip=True)

        # 5. Safety Check & Save
        if len(car_data) > 2: # URL + Price + at least one spec
            with open(output_file, "a") as out:
                out.write(json.dumps(car_data) + "\n")
            print(f"Saved: {car_data.get('Price', 'N/A')} | {len(car_data)-2} specs.")
        else:
            print("WARNING: Data missing. Check if page loaded correctly.")

    except Exception as e:
        print(f"Error at {link}: {e}")
        browser.quit()
        browser = get_browser()
        time.sleep(1) 
        continue

browser.quit()

Scraping 1/1101: https://www.cardekho.com/used-car-details/used-Land-rover-range-rover-30-diesel-lwb-vogue-cars-Gurgaon_e77c1e2c-af8c-4db4-858f-18a99797dc7b.htm
Saved: ₹1.40 Crore | 51 specs.
Scraping 2/1101: https://www.cardekho.com/used-car-details/used-Kia-seltos-x-line-turbo-dct-cars-Greater-Noida_3b9d9e62-0e20-41f7-95c9-dfe6779ca8b4.htm
Scraping 3/1101: https://www.cardekho.com/used-car-details/used-Hyundai-grand-i10-nios-magna-cars-New-Delhi_d76d88ed-f1a3-468c-b61e-e791d4a79a4a.htm
Scraping 4/1101: https://www.cardekho.com/used-car-details/used-Kia-seltos-gtx-plus-dct-cars-Gurgaon_c18b1275-3f1f-4d98-aeb2-4ad8ada1d907.htm
Saved: ₹8.50 Lakh | 42 specs.
Scraping 5/1101: https://www.cardekho.com/used-car-details/used-Kia-sonet-htx-turbo-imt-bsvi-cars-Greater-Noida_32af021a-a93b-4165-9de8-5509abb9a2ab.htm
Saved: ₹6.65 Lakh | 42 specs.
Scraping 6/1101: https://www.cardekho.com/used-car-details/used-Mahindra-thar-lx-4wd-hard-top-bsvi-cars-New-Delhi_1b0de125-4ca7-41c9-8de4-afbb712af27c.h

KeyboardInterrupt: 